In [1]:
from keras.applications import VGG16
#VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 64
img_cols = 64
#Load the VGG16 model 
model = VGG16(weights = 'imagenet' ,
             include_top = False ,
             input_shape = (img_rows ,img_cols , 3))

Using TensorFlow backend.


In [2]:
#Now print our layers
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [3]:
def addTopModel(bottom_model , num_classes):
    """creates the top or the head of the model that will be
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation ='relu')(top_model)
    top_model = Dense(512,activation ='relu')(top_model)
    top_model = Dense(248,activation ='relu')(top_model)
    top_model = Dense(num_classes,activation ='softmax')(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense ,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
num_classes = 5

FC_head =addTopModel(model, num_classes)
modelnew= Model(inputs =model.input ,outputs =FC_head)
print(modelnew.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir ='C:/Users/Prince Raj/Desktop/task4/train/'
validation_data_dir ='C:/Users/Prince Raj/Desktop/task4/val/'
    
train_datagen = ImageDataGenerator(
      rescale =1./255,
      rotation_range =20,
      width_shift_range = 0.2,
      horizontal_flip = True,
      fill_mode ='nearest')

validation_datagen = ImageDataGenerator(rescale =1.255)
                 
#change the batchsize according to your system RAM
train_batchsize =15
val_batchsize =10

train_generator = train_datagen.flow_from_directory(
         train_data_dir ,
         target_size =(img_rows,img_cols),
         batch_size = train_batchsize ,
         class_mode ='categorical')

validation_generator = validation_datagen.flow_from_directory(
          validation_data_dir,
          target_size = (img_rows,img_cols),
          batch_size =val_batchsize ,
          class_mode = 'categorical',
          shuffle = False)
         

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [6]:
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint ,EarlyStopping

checkpoint= ModelCheckpoint("data.h5",
                            monitor= "val_loss",
                            
                            mode ="min",
                            save_best_only =True,
                            verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)

#we will going to put our call backs into callback list
callbacks = [checkpoint,earlystop]

#Now we use a very small lerarning rate
modelnew.compile(loss ='binary_crossentropy',
                optimizer =Adam(lr=0.001),
                metrics=['accuracy'])
nb_train_samples =91
nb_validation_samples =24
epochs =10 
batch_size =10
history=modelnew.fit_generator(train_generator,
                            
    steps_per_epoch=nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
modelnew.save("data.h5")
                            

Epoch 1/10
9/9 [==============================] - 18s 2s/step - loss: 0.7609 - accuracy: 0.7659 - val_loss: 6.1348 - val_accuracy: 0.7000

Epoch 00001: val_loss improved from inf to 6.13484, saving model to data.h5
Epoch 2/10
9/9 [==============================] - 15s 2s/step - loss: 0.5156 - accuracy: 0.8000 - val_loss: 6.1348 - val_accuracy: 0.6000

Epoch 00002: val_loss did not improve from 6.13484
Epoch 3/10
9/9 [==============================] - 17s 2s/step - loss: 0.5082 - accuracy: 0.8000 - val_loss: 2.6639e-04 - val_accuracy: 0.7333

Epoch 00003: val_loss improved from 6.13484 to 0.00027, saving model to data.h5
Epoch 4/10
9/9 [==============================] - 15s 2s/step - loss: 0.5674 - accuracy: 0.7874 - val_loss: 0.6398 - val_accuracy: 0.6800

Epoch 00004: val_loss did not improve from 0.00027
Epoch 5/10
9/9 [==============================] - 14s 2s/step - loss: 0.5142 - accuracy: 0.8000 - val_loss: 3.0271 - val_accuracy: 0.7333

Epoch 00005: val_loss did not improve from 

In [7]:
from keras.models import load_model
from keras.preprocessing import image
classifier=load_model('data.h5')
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile ,join
from random import *

dict ={"[0]":"Ben Afflek",
           "[1]":"Elton John",
           "[2]":"Jerry Seinfeld",
           "[3]":"Madonna",
           "[4]":"Mindy Kaling"}
dict_n ={"ben_afflek":"Ben Afflek",
        "elton_john":"Elton John",
        "jerry_seinfeld":"Jerry Seinfeld",
        "madonna":"Madonna",
        "mindy_kaling":"Mindy kaling"}
def draw_test(name,pred,im):
    Celebrity = dict[str(pred)]
    BLACK =[0,0,0]
    expand_image=cv2.CopyMakeBorder(im,80,0,0,100,cv2.BORDER_CONSTANT, value =BLACK)
    cv2.putText(expanded_image,Celebrity,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow(name,expanded_image)
    
def getImage(path):
    """function load a random images from a random folder in our test path"""
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path,x)) ,os.listdir(path)))
    random_directory =np.random.randint(0,len(folders))
    path_class=folders[random_directory]
    print("class - " + dict_n[str(path_class)])
    
    file_path=path + path_class
    file_names =[f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index=np.random.randint(0 ,len(file_names))
    image_name =file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)

for i in range (0,10):
    input_im = getImage("C:/Users/Prince Raj/Desktop/task4/val/")
    input_original =input_im.copy()
    input_original =cv2.resize(input_original,None,fx=0.5,fy=0.5,interpolation =cv2.INTER_LINEAR)
    
    input_im =cv2.resize(input_im,(64,64),interpolation=cv2.INTER_LINEAR)
    input_im = input_im / 225.
    input_im =input_im.reshape(1,64,64,3)
    
    #Get prediction
    res =np.argmax(classifier.predict(input_im,1,verbose=0), axis =1)
    
    #Show image with predicted class
    draw_test=("prediction",res ,input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()    

class - Ben Afflek
class - Ben Afflek
class - Ben Afflek
class - Jerry Seinfeld
class - Ben Afflek
class - Ben Afflek
class - Ben Afflek
class - Ben Afflek
class - Ben Afflek
class - Elton John
